## Ønsker å undersøke havnivådata
API-en inneholder mange datasett med klimadata, som spenner over ulike typer målinger og geografiske områder. Filtrerer derfor ut datasettene som ikke inneholder SSH (Sea Surface Height) og som ikke er over datamengde terskelen (tilecount>=10000). 

In [7]:
import sys
sys.path.append("../src")

from Havnivådata.Datainnsamling import filtrer_ssh_metadata

df_ssh = filtrer_ssh_metadata(terskel=10000)
df_ssh.head()




,title,tileCount,iso_start,iso_end
0,NASA_SSH_REF_SIMPLE_GRID_V1_Monthly,83381,1992-10-01T00:00:00+0000,2024-11-01T00:00:00+0000
1,SSHIBC_ECCO_version4_release4,78624,1992-01-16T18:00:00+0000,2017-12-16T06:00:00+0000
2,SSHNOIBC_ECCO_version4_release4,78624,1992-01-16T18:00:00+0000,2017-12-16T06:00:00+0000
3,SSH_ECCO_version4_release4,78624,1992-01-16T18:00:00+0000,2017-12-16T06:00:00+0000


### Finner SSH datasettene ved bruk av list comprehensions
Dette går gjennom hele datasettet (som er en liste med ordbøker) og lager en ny liste som kun inneholder de datasettene med "SSH" i shortName. Det er en effektiv og lesbar måte å filtrere og hente ut relevante datasett på. 


In [8]:
from pathlib import Path
import sys

# Finn prosjektroten (ett nivå opp fra notebooks/)
project_root = Path.cwd().parent

# Legg til src/Havnivådata i sys.path for import
src_path = project_root / "src" / "Havnivådata" 
sys.path.append(str(src_path))

from Datainnsamling import ssh_datasets_listcomp

metadata_path = project_root / "data" / "Havnivådata" / "metadata_datasetliste.json"
df_ssh = ssh_datasets_listcomp(str(metadata_path))
df_ssh.head()







,title,tileCount,iso_start,iso_end
0,NASA_SSH_REF_SIMPLE_GRID_V1_Monthly,83381,1992-10-01T00:00:00+0000,2024-11-01T00:00:00+0000
1,SSHIBC_ECCO_version4_release4,78624,1992-01-16T18:00:00+0000,2017-12-16T06:00:00+0000
2,SSHNOIBC_ECCO_version4_release4,78624,1992-01-16T18:00:00+0000,2017-12-16T06:00:00+0000
3,SSH_ECCO_version4_release4,78624,1992-01-16T18:00:00+0000,2017-12-16T06:00:00+0000


### Anlyserer SSH datasettene ved bruk av Pandas SQL
Pandasql lar deg bruke SQL-spørringer direkte på Pandas DataFrames. Dette kan gjøre det enklere og mer intuitivt å jobbe med data for de som er godt kjent med SQL. I stedet for å måtte bruke flere forskjellige Pandas-funksjoner når du skal filtrere, sortere eller gruppere data, kan dette gjøres med én SQL-setning. Dette gir en mer oversiktlig kode, spesielt på kompliserte operasjoner. Pandasql gjør det lettere å kombinere flere DataFrames med JOIN, noe som ofte er litt vanskeligere med vanlige Pandas-funksjoner.

In [9]:
from pathlib import Path
import sys

# Finn prosjektroten (ett nivå opp fra notebooks/)
project_root = Path.cwd().parent

# Legg til src/Havnivådata i sys.path for import
src_path = project_root / "src" / "Havnivådata"
sys.path.append(str(src_path))

from Datainnsamling import filtrer_ssh_datasets_sql

metadata_path = project_root / "data" / "Havnivådata" / "metadata_datasetliste.json"
df_ssh_sql = filtrer_ssh_datasets_sql(str(metadata_path))
df_ssh_sql.head()



,title,tileCount,iso_start,iso_end
0,NASA_SSH_REF_SIMPLE_GRID_V1_Monthly,83381,1992-10-01T00:00:00+0000,2024-11-01T00:00:00+0000
1,SSHIBC_ECCO_version4_release4,78624,1992-01-16T18:00:00+0000,2017-12-16T06:00:00+0000
2,SSHNOIBC_ECCO_version4_release4,78624,1992-01-16T18:00:00+0000,2017-12-16T06:00:00+0000
3,SSH_ECCO_version4_release4,78624,1992-01-16T18:00:00+0000,2017-12-16T06:00:00+0000


### Velger å undersøke "NASA_SSH_REF_SIMPLE_GRID_V1_Monthly" videre
Ser i tabellen at "NASA_SSH_REF_SIMPLE_GRID_V1_Monthly" har største mengden data og målinger over den lengste perioden. Derfor vil det være naturlig å jobbe med dette datasettet videre. Laster ned datasettet som "havnivaadata.json" under data / Havnivådata / for videre behandling.

In [10]:
from pathlib import Path
import sys

project_root = Path.cwd().parent
src_path = project_root / "src" / "Havnivådata"
sys.path.append(str(src_path))

from Datainnsamling import last_ned_nasa_ssh_data

df_nasa = last_ned_nasa_ssh_data()
df_nasa.head()



✅ Data lagret i: C:\Users\madsj\OneDrive\Dokumenter\tdt4114\TDT4114-Prosjekt\data\Havnivådata\havnivaadata.json


,min,mean,max,time,std,iso_time,cnt
0,-0.388882,-0.000171,0.431853,717897600,0.056767,1992-10-01T00:00:00+0000,145539
1,-0.366509,-0.004225,0.296866,720576000,0.050972,1992-11-01T00:00:00+0000,147346
2,-0.311839,-0.012979,0.319928,723168000,0.050031,1992-12-01T00:00:00+0000,147965
3,-0.330625,-0.018975,0.358001,725846400,0.055296,1993-01-01T00:00:00+0000,147545
4,-0.362726,-0.020470,0.390943,728524800,0.059630,1993-02-01T00:00:00+0000,147413


### Undersøker manglende verdier i datasettet og fyller inn ved bruk av valgt metode
Her brukes rens-funksjonen til å fylle inn manglende verdier. Funksjonen kan bruke lineær interpolasjon, naboverdier, gjennomsnitt, median eller fjerne rader med manglende verdier. Alt avhengig av hvilken metode som velges. 


In [4]:
from pathlib import Path
import sys
import pandas as pd

# Sett opp src-sti for import av egne funksjoner
project_root = Path.cwd().parent
src_path = project_root / "src" / "Havnivådata"
sys.path.append(str(src_path))

from Databehandling import rens_manglende_verdier

# 1. Les inn datasettet ditt
df = pd.read_json(project_root / "data" / "Havnivådata" / "havnivaadata.json", lines=True)

# 2. Rens manglende verdier – bruk "mean", "median", "drop","bfill", "interpolate", eller "ffill" utifra hva vi velger nedenfor
print("\n--- Før rensing ---")
print(df.isnull().sum())

df_renset = rens_manglende_verdier(df, metode="interpolate")  # Du kan bytte 

print("\n--- Etter rensing ---")
print(df_renset.isnull().sum())

print("\nEksempel på rader etter rensing:")
print(df_renset.head())





--- Før rensing ---
time        0
cnt         0
std         0
max         0
mean        0
min         0
iso_time    0
dtype: int64
Antall manglende verdier per kolonne:
 time    0
cnt     0
std     0
max     0
mean    0
min     0
dtype: int64
Manglende verdier er fylt inn med lineær interpolasjon.

--- Etter rensing ---
time        0
cnt         0
std         0
max         0
mean        0
min         0
iso_time    0
dtype: int64

Eksempel på rader etter rensing:
        time     cnt       std       max      mean       min  \
0  717897600  145539  0.056767  0.431853 -0.000171 -0.388882   
1  720576000  147346  0.050972  0.296866 -0.004225 -0.366509   
2  723168000  147965  0.050031  0.319928 -0.012979 -0.311839   
3  725846400  147545  0.055296  0.358001 -0.018975 -0.330625   
4  728524800  147413  0.059630  0.390943 -0.020470 -0.362726   

                   iso_time  
0 1992-10-01 00:00:00+00:00  
1 1992-11-01 00:00:00+00:00  
2 1992-12-01 00:00:00+00:00  
3 1993-01-01 00:00:00+00:00

Vi ser ovenfor at datasettet ikke har noen manglende verdier.

## Demonstrasjon av håndtering av manglende verdier (NaN)
For å illustrere hvordan rens-funksjonen håndterer manglende verdier, opprettes et testdatasett der NaN-verdier legges inn i tilfeldige rader og kolonner. Deretter anvendes rens-funksjonen med ulike metoder (f.eks. interpolasjon, gjennomsnitt eller fjerning av rader) for å fylle inn eller fjerne de manglende verdiene.

Resultatet viser:
- Hvilke rader som fikk NaN og hvordan de ser ut før rensing.
- Hvordan de samme radene ser ut etter rensing.
- For metoden "drop" vises hvilke rader som faktisk blir fjernet fordi de hadde manglende verdier. 


In [1]:
import sys
import pandas as pd
from pathlib import Path

# Sett opp sti til src for import av egne funksjoner
project_root = Path.cwd().parent
src_path = project_root / "src" / "Havnivådata"
sys.path.append(str(src_path))

from Databehandling import demonstrer_rensing_av_manglende_verdier

df = pd.read_json(project_root / "data" / "Havnivådata" / "havnivaadata.json", lines=True)
df_renset = demonstrer_rensing_av_manglende_verdier(df, antall_nan_per_kolonne=3, metode="interpolate") # Her kan vi endre antall NaN og metode



La til NaN i kolonne 'time' på rader: [336, 307, 90]
La til NaN i kolonne 'cnt' på rader: [142, 324, 97]
La til NaN i kolonne 'std' på rader: [95, 168, 15]
La til NaN i kolonne 'max' på rader: [124, 249, 156]
La til NaN i kolonne 'mean' på rader: [343, 95, 51]
La til NaN i kolonne 'min' på rader: [13, 310, 100]

Antall manglende verdier etter tilsetning:
time        3
cnt         3
std         3
max         3
mean        3
min         3
iso_time    0
dtype: int64

--- Rader med NaN FØR rensing ---
             time       cnt       std       max      mean       min  \
13   7.521120e+08  146998.0  0.050383  0.329535 -0.008982       NaN   
15   7.573824e+08  147377.0       NaN  0.253833 -0.012013 -0.251388   
51   8.520768e+08  147963.0  0.055375  0.275407       NaN -0.634660   
90            NaN  148197.0  0.062357  0.350955 -0.000877 -0.404227   
95   9.677664e+08  145604.0       NaN  0.477461       NaN -0.253206   
97   9.730368e+08       NaN  0.055201  0.427581  0.014957 -0.262881   


### Fjerning og utfylling av usannsynlige verdier i datasettet
Det forventes uregelmessigheter i datasettet, og derfor er det nødvendig med datarensing. Først identifiseres verdier som ligger utenfor et bestemt intervall. Disse settes til NaN. Deretter benyttes rens-funksjonen med valgt metode for å fylle inn de manglende verdiene. Dette sikrer at datasettet blir realistisk og komplett før videre analyse.

In [6]:

from pathlib import Path
import sys
import pandas as pd

# Sett opp sti til src for import av egne funksjoner
project_root = Path.cwd().parent
src_path = project_root / "src" / "Havnivådata"
sys.path.append(str(src_path))

from Databehandling import sett_usannsynlige_til_nan, rens_manglende_verdier

# Les inn datasettet
df = pd.read_json(project_root / "data" / "Havnivådata" / "havnivaadata.json", lines=True)

# Sett grenser for rimelige verdier i datasettet (i meter)
kolonnegrenser = {
    "mean": [-0.3, 0.3],
    "max": [-1, 1],
    "min": [-1, 1]
}

# 1. Sett usannsynlige verdier til NaN (men behold radene)
df_with_nan = sett_usannsynlige_til_nan(df, kolonnegrenser)

# 2. Fyll inn de NaN-ene med ønsket metode
df_renset = rens_manglende_verdier(df_with_nan, metode="interpolate")  # alle metodene i rens-funksjonen fungerer





  Satt 0 usannsynlige verdier til NaN i 'mean' (utenfor [-0.3, 0.3])
  Satt 0 usannsynlige verdier til NaN i 'max' (utenfor [-1, 1])
  Satt 0 usannsynlige verdier til NaN i 'min' (utenfor [-1, 1])
Antall manglende verdier per kolonne:
 time    0
cnt     0
std     0
max     0
mean    0
min     0
dtype: int64
Manglende verdier er fylt inn med lineær interpolasjon.


For variablene «min» og «max» er nedre og øvre grenser satt til henholdsvis −1 meter og 1 meter, mens variabelen «mean» er begrenset mellom −0,3 og 0,3 meter. Disse intervallene sikrer at alle data ligger innenfor realistiske verdier. Disse kan alltid endres på utifra hva brukeren ønsker. Variablene «min» og «max» representerer de laveste og høyeste havnivåmålingene registrert hver måned på en global skala (omtrent 85-90% av verdenshavene), og derfor tillates et større intervall. Variabelen «mean», som er månedlig gjennomsnitt av alle målinger globalt, forventes å variere mindre og er derfor gitt et redusert intervall.